In [19]:
from transformers import AutoTokenizer,DataCollatorForLanguageModeling
from datasets import Dataset, load_dataset
from collections import Counter
import re
from nltk.util import ngrams
from multiprocessing import Pool, cpu_count
import itertools
import pickle
import flashtext
import glob
import psutil

In [2]:
dir_train = 'gutenberg/gpt2_training_gutenberg_train.txt'
dir_test = 'gutenberg/gpt2_training_gutenberg_test.txt'

In [2]:
dataset = load_dataset("cc100", lang="de")

Using custom data configuration de-lang=de
Reusing dataset cc100 (/home/andreas/.cache/huggingface/datasets/cc100/de-lang=de/0.0.0/b583dd47b0dd43a3c3773075abd993be12d0eee93dbd2cfe15a0e4e94d481e80)


In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'text'],
        num_rows: 264959282
    })
})

In [4]:
text = dataset['train'][:10000000]['text']

In [7]:
dataset.num_rows

{'train': 264959282}

In [4]:
def preprocess_text(text):
    return re.sub('[^a-zäöüß ]', ' ', ' '.join(text).lower())

def prep_count(text):
    tokens = preprocess_text(text).split()
    #list(ngrams(tokens,2))
    return (tokens)

In [5]:
step = 1000000
workers = min(8,int(cpu_count()-10))
j = 0
while j < dataset.num_rows['train']+step*workers:
    with Pool(workers) as p:

        results = p.map(prep_count,[dataset['train'][start:start+step]['text'] for start in range(j,j+step*workers,step)])

    results = list(itertools.chain.from_iterable(results))

    with open('cc_100_vocab/count_'+str(j)+'.pkl', 'wb') as handle:
        pickle.dump(Counter(results), handle, protocol=pickle.HIGHEST_PROTOCOL)

    with open('cc_100_vocab/text_'+str(j)+'.txt', 'w') as f:
        f.write(' '.join(results))

    del results
    j+= step*workers
    print(j)



8000000
16000000
24000000
32000000
40000000
48000000
56000000
64000000
72000000
80000000
88000000
96000000
104000000
112000000
120000000
128000000
136000000
144000000
152000000
160000000
168000000
176000000
184000000
192000000
200000000
208000000
216000000
224000000
232000000
240000000
248000000
256000000
264000000
272000000
280000000


In [16]:
text_path = 'cc_100_vocab'


In [17]:
files = glob.glob(text_path+"/*.txt")

In [18]:
files

['cc_100_vocab/text_80000000.txt',
 'cc_100_vocab/text_192000000.txt',
 'cc_100_vocab/text_104000000.txt',
 'cc_100_vocab/text_256000000.txt',
 'cc_100_vocab/text_88000000.txt',
 'cc_100_vocab/text_232000000.txt',
 'cc_100_vocab/text_216000000.txt',
 'cc_100_vocab/text_176000000.txt',
 'cc_100_vocab/text_160000000.txt',
 'cc_100_vocab/text_56000000.txt',
 'cc_100_vocab/text_152000000.txt',
 'cc_100_vocab/text_272000000.txt',
 'cc_100_vocab/text_48000000.txt',
 'cc_100_vocab/text_40000000.txt',
 'cc_100_vocab/text_136000000.txt',
 'cc_100_vocab/text_208000000.txt',
 'cc_100_vocab/text_112000000.txt',
 'cc_100_vocab/text_168000000.txt',
 'cc_100_vocab/text_0.txt',
 'cc_100_vocab/text_16000000.txt',
 'cc_100_vocab/text_224000000.txt',
 'cc_100_vocab/text_184000000.txt',
 'cc_100_vocab/text_24000000.txt',
 'cc_100_vocab/text_240000000.txt',
 'cc_100_vocab/text_96000000.txt',
 'cc_100_vocab/text_120000000.txt',
 'cc_100_vocab/text_128000000.txt',
 'cc_100_vocab/text_248000000.txt',
 'cc_100

In [15]:
text = ''
for file in files:
    with open(file) as f:
        text += ' ' + f.read()

    print(psutil.virtual_memory().percent)
    if psutil.virtual_memory().percent > 80:
        break

'cc_100_vocab/text_0.txt/*.txt'

In [21]:
psutil.virtual_memory().percent

16.2

In [3]:
from flashtext import KeywordProcessor
keyword_processor = KeywordProcessor()
keyword_processor.add_keyword('die wir uns solcher gnade freun')

True

In [4]:

keywords_found = keyword_processor.extract_keywords(text)


In [11]:
text += 'die wir uns solcher gnade freun'

In [12]:
re.findall('die wir uns solcher gnade freun',text)

['die wir uns solcher gnade freun']